In [2]:
import json
import pandas as pd

In [98]:
gemma = pd.DataFrame(json.load(open('/home/student/katya/tbank-ml-camp-sirius/data/validation_gemma.json')))
llama = pd.DataFrame(json.load(open('/home/student/katya/tbank-ml-camp-sirius/data/validation_llama.json')))
qwen = pd.DataFrame(json.load(open('/home/student/katya/tbank-ml-camp-sirius/data/validation_qwen.json')))

In [99]:
gemma.shape, llama.shape, qwen.shape # why

((3064, 3), (3064, 3), (3100, 3))

In [100]:
def check(set_array_labels, array_labels):
    for j, label in enumerate(set_array_labels):
        count = array_labels.count(label)
        if count >= 2:
            return label
        if  j == 2:
            return "oos"

In [101]:
new_results = []
for i, row_qwen in qwen.iterrows():
    query_qwen = row_qwen['query']
    row_llama, row_gemma = llama[llama['query'] == query_qwen], gemma[gemma['query'] == query_qwen]
    label_llama, label_gemma, label_qwen = row_llama['predicted'].values, row_gemma['predicted'].values, row_qwen['predicted label']
    if len(label_llama) != 0:
        label_llama, label_gemma = label_llama[0], label_gemma[0]
        array_labels = [label_llama, label_gemma, label_qwen]
        set_array_labels = list(set(array_labels))
        label_ens = check(set_array_labels, array_labels)
    else:
        label_ens = label_qwen
            
    new_results.append({'query': query_qwen, 'initial': row_qwen['initial label'], 'predicted': label_ens})

In [102]:
with open('/home/student/katya/tbank-ml-camp-sirius/data/merged_results_validation.json', "w") as f:
    json.dump(new_results, f, indent=4)

In [103]:
len(new_results)

3100

In [104]:
df = pd.DataFrame(new_results)
df['predicted'].value_counts()

predicted
oos                      143
shopping_list_update      31
restaurant_suggestion     27
todo_list_update          27
oil_change_when           26
                        ... 
schedule_maintenance      14
how_old_are_you           14
bill_balance              13
todo_list                 12
shopping_list             11
Name: count, Length: 151, dtype: int64

In [106]:
full_train = pd.DataFrame(json.load(open('/home/student/katya/tbank-ml-camp-sirius/data/merged_results_train.json')))
full_train['predicted'].value_counts()

predicted
oos                      500
restaurant_suggestion    128
update_playlist          127
who_made_you             125
shopping_list_update     125
                        ... 
sync_device               64
order                     64
play_music                62
next_holiday              61
where_are_you_from        60
Name: count, Length: 151, dtype: int64

In [111]:
for i in full_train['predicted'].unique():
    if i not in df['predicted'].unique():
        print(i)